In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create a Spark session
spark = SparkSession.builder.appName("test").getOrCreate()

25/06/07 14:52:10 WARN Utils: Your hostname, Rams-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
25/06/07 14:52:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/07 14:52:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
data = [
    (101, "ramkrishnaa32@gmail.com"),
    (102, "ramkrishnaa_32@gmail.com"),
    (103, "ramkrishna.achary@gmail.com")
]

In [ ]:
schema = StructType([
    StructField("Id", IntegerType(), True),
    StructField("email", StringType(), True)
])

In [ ]:
# Create DataFrame
df = spark.createDataFrame(data, schema)
df.show(truncate=False)

In [ ]:
# Split the email into username and domain
df = df.withColumn("username", split(col("email"), "@").getItem(0)) \
       .withColumn("domain", split(col("email"), "@").getItem(1))

df.show(truncate=False)

In [ ]:
df = df.withColumn("masked_email", 
                   expr("repeat('*', length(username)) || '@' || domain"))
df.show(truncate=False)

In [ ]:
df_new = df.withColumn("masked_email", expr("""
    CASE 
        WHEN length(username) <= 5 THEN repeat('*', length(username))
        ELSE concat(
            substring(username, 1, 2),
            repeat('*', length(username) - 3),
            substring(username, -2),
            '@',
            domain
        )
    END
"""))

df_new.show(truncate=False)